In [1]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.4/730.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.3 MB/s eta 0:00:00


In [2]:

!pip install --upgrade autogen autogen-agentchat==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.6 MB/s eta 0:00:00
Reason for being yanked: This version accidentally included an unintentional GPL dependency. Please upgrade to 0.4.2.
Reason for being yanked: This version accidentally included an unintentional GPL dependency. Please upgrade to 0.4.2.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires p

In [3]:
import autogen

In [4]:
autogen.__version__

'0.8.5'

In [7]:
from autogen import config_list_from_json

config_list_gpt4 = config_list_from_json(
    "OAI_CONFIG_LIST.json")

For json file format, refer to https://microsoft.github.io/autogen/0.2/docs/topics/llm_configuration/

below is the sample format

[
  {
    "model": "gpt-4o-mini",
    "api_key": "sk-jfsdjf843925jjfasd"
  }
]

In [8]:
gpt4_config = {
    "cache_seed": 42,
    "temperature": 0.0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

In [9]:
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager

In [10]:
# Define Agents
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "coding"},
)

dataprep = autogen.AssistantAgent(
    name="dataprep",
    llm_config=gpt4_config,
    system_message="You are an expert in data preparation. You will receive data and instructions, and will produce cleaned, transformed data.",
)

edaagent = autogen.AssistantAgent(
    name="edaagent",
    llm_config=gpt4_config,
    system_message="You are an expert in exploratory data analysis (EDA). You will receive data and perform EDA, creating visualizations and summaries.",
)

reportgenerator = autogen.AssistantAgent(
    name="reportgenerator",
    llm_config=gpt4_config,
    system_message="You are an expert in report generation. You will receive data and analysis results, and create a comprehensive report.",
)

executor = autogen.AssistantAgent(
    name="executor",
    llm_config=gpt4_config,
    system_message="You are a code execution expert. You are able to execute python code and provide the results. If code fails, please provide the error message.",
    code_execution_config={"work_dir": "coding"},
)

critic = autogen.AssistantAgent(
    name="critic",
    llm_config=gpt4_config,
    system_message="You are a critic. You review the work of other agents and provide feedback on accuracy, completeness, and clarity. You ask clarifying questions and help the other agents improve their work.",
)

# Define Conversation Flow
groupchat = autogen.GroupChat(
    agents=[user_proxy, dataprep, edaagent, reportgenerator, executor, critic], messages=[], max_round=20
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)


# Get dataset input from user
print("Please enter your dataset as a CSV string (e.g., 'age,income,city\\n25,50000,New York\\n30,60000,London'):")
dataset_input = input()

# Start Conversation with user input
user_proxy.initiate_chat(
    manager,
    message=f"Prepare and analyze this data:\n{dataset_input}\nClean the data, perform EDA, and generate a report. If you need to execute code, ask the executor.",
)

Please enter your dataset as a CSV string (e.g., 'age,income,city\n25,50000,New York\n30,60000,London'):
'age,income,city\n25,50000,New York\n30,60000,London'
user_proxy (to chat_manager):

Prepare and analyze this data:
'age,income,city\n25,50000,New York\n30,60000,London'
Clean the data, perform EDA, and generate a report. If you need to execute code, ask the executor.

--------------------------------------------------------------------------------

Next speaker: dataprep

dataprep (to chat_manager):

The data provided is quite small and clean. However, let's go through the steps of data preparation and analysis.

1. **Data Cleaning**: The data seems to be already clean. There are no missing values or incorrect entries. The data types for each column are also appropriate (age and income are numerical, city is categorical).

2. **Exploratory Data Analysis (EDA)**: Given the small size of the data, there's not much we can do in terms of EDA. However, we can still provide some basic st

ChatResult(chat_id=None, chat_history=[{'content': "Prepare and analyze this data:\n'age,income,city\\n25,50000,New York\\n30,60000,London'\nClean the data, perform EDA, and generate a report. If you need to execute code, ask the executor.", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "The data provided is quite small and clean. However, let's go through the steps of data preparation and analysis.\n\n1. **Data Cleaning**: The data seems to be already clean. There are no missing values or incorrect entries. The data types for each column are also appropriate (age and income are numerical, city is categorical).\n\n2. **Exploratory Data Analysis (EDA)**: Given the small size of the data, there's not much we can do in terms of EDA. However, we can still provide some basic statistics:\n\n   - Age: The minimum age is 25, the maximum age is 30.\n   - Income: The minimum income is 50000, the maximum income is 60000.\n   - City: There are two unique cities, New York and London.\n\n3

In [ ]:
filename = "./sample_data/mnist_test.csv"

Disclaimer: These capabilities are still in their experimental, phase, the table generated below suggested links that do not match what is being saif (hallucinated).